In [1]:
def HelloClass():
    return "Hello Class!"

print(HelloClass())

Hello Class!


# Lab 5 

I know it seems like just yesterday you were struggling with "what is a notebook? what is github? how do I `import arcgis`??" But, that's the nature of the quarter system and here we are, ready to start putting things all together.

If you remember the core goals of this class, they're to help you programmatically:
1. Get some data.
2. Format, process, analyze, etc. said data.
3. Visualize it in some way.

We've done so primarily through the ArcGIS API for Python **and** more recently, GeoPandas and Folium. In this lab, we're going to **acquire** data from Twitter (a robust, well developed API that spits out a lot of data), process it into a format that's easy to map (in this case, I recommend a csv), and then add it to a series of interactive maps. 

As this is your second to last lab, I'm going to leave open precisely how you do a lot of this; however, I'm also going to be working more directly with each group throughout the process. What I mean here is that I recommend you use Folium for your visualization, but you can use the ArcGIS API if you prefer.

**In fact, you can and should mix and match whatever _tools_ work best for your group.** After this class, you'll encounter many problems - use the core ideas and tools you've encountered in this class to cobble together clever solutions! Iteration! Flow control! Using libraries and reading documentation are the keys to solving a vast, vast variety of geospatial problems.


### Building our environment

As always, let's get out virtual environment set up before we start. This time, I'm going to be adding in installing something from a different channel so that we can install everything all at once. I do so with the `-c` part of the lines below.

`conda create -n lab5 python=3.6`

After that's done, activate your environment and then install the necessary packages. 

`conda install -c conda-forge geopandas jupyter folium fiona tweepy geopy`

I'm assuming you *aren't* going to use the ArcGIS API for Python. If you *are*, you can install it in your created environment with:

`conda install -c esri arcgis` *(See how you switch 'channels' to grab different packages?)*

You should recognize most of the packages above as we've used them before; I'm adding in tweepy (which is a library to interface Twitter's API) and geopy (which is a handy library for geocoding).

### Let's look at some tweets!

Now, *in the future*, if you want to do this you'll have to create your own twitter account and apply to become a developer. Their Standard APIs are still free to use and test, but they make you apply and try to sell you their more robust ones. You can [read about that all here](https://developer.twitter.com/en/apply-for-access).

But, for this lab, I'm going to let you use some of my own API keys. To access the Twitter API, you need a set of **keys and access tokens**. There are steps at the [developers page](https://developer.twitter.com) that walk you through this, but *for now* know that you'll need a: Consumer Key, a Consumer Secret Key, an Access Token, and an Access Token Secret. 

We talked a bit about this in our API lecture - these are basically like log-ins and passwords that let the API owner keep track of who is accessing their information, how much of it, and when. 

Below, I've supplied each group with a set of these tokens. Please note **storing keys like this in a github account is TERRIBLE PRACTICE**. You always want to remove private information before you upload, but these are basic level open information and it's just easiest for this class right now.

Let's check if everything is working. In the following cell, assign each key to a variable, and then pass those variables into a Tweepy API object.

In [1]:
import tweepy

ConsumerKey = 'yj50FC54j6ONZqxS5IzGnLdbF'
ConsumerSecret = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu'
AccessKey = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1'
AccessSecret = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'

auth = tweepy.OAuthHandler(ConsumerKey, ConsumerSecret)
auth.set_access_token(AccessKey, AccessSecret)

api = tweepy.API(auth)


print(api.user_timeline(id='marxbot1', count=1)) #This simply pulls the last tweet from an account

[Status(_api=<tweepy.api.API object at 0x0000024FA15BC828>, _json={'created_at': 'Sat May 05 15:33:39 +0000 2018', 'id': 992789430262886400, 'id_str': '992789430262886400', 'text': '@alogicalfallacy A change in commodity A may therefore be imagined that all Catholics can be popes together.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'alogicalfallacy', 'name': 'Jim Thatcher', 'id': 19347325, 'id_str': '19347325', 'indices': [0, 16]}], 'urls': []}, 'source': '<a href="http://twitter.com/marxbot1" rel="nofollow">Marxbot1</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 19347325, 'in_reply_to_user_id_str': '19347325', 'in_reply_to_screen_name': 'alogicalfallacy', 'user': {'id': 830247048414842880, 'id_str': '830247048414842880', 'name': 'Marxbot1', 'screen_name': 'marxbot1', 'location': '', 'description': "I'm a markov chain based bot trained on Marx's works. I'll tweet on my own or respon

### If that ran successfully, you should have a giant mess of text.

That's the data that accompanies a single tweet. Interesting, huh? Check out the [reference docs](http://tweepy.readthedocs.io/en/v3.5.0/api.html#) for tweepy and spend some time experimenting if you want.

Here, I'll pull the same tweet as above, but this time I'm **only** going to print out the text property of the Status object and then check for some location information.



In [2]:
#Note: This calls the api object that we created in the previous cell.
#THAT MEANS THE PREVIOUS CELL HAS TO RUN BEFORE THIS ONE.

#Also note: First I specify which list object I want, then I pull a property from it.

print(api.user_timeline(id='marxbot1', count=1)[0].text)

#Now, let's see if there's some lat and long associated with the tweet
print(api.user_timeline(id='marxbot1', count=1)[0].geo)
print(api.user_timeline(id='marxbot1', count=1)[0].coordinates)

@alogicalfallacy A change in commodity A may therefore be imagined that all Catholics can be popes together.
None
None


### (Un)fortunately, most tweets don't actually have location information associated with them. 

There's been *a lot* written about this and the numbers vary from under 5% to 20% or so of tweets. Additionally, it's been argued that upwards of 60% of tweets *can* have some location inferred due to language use, topic, etc.

That's all interesting (and please do email me for citations if you so desire); however, it's also kind of besides the point here. We're interested in learning how to interact with APIs and process data, we can argue about the ephemerality of said data another day.

Let's query some topic of interest and see if we can find some spatial data.


In [3]:
#We're going to set up a couple of tricks here
#We're going to recreate our api with a few new settings.

import json
# I'm going to use python's built in json library to parse the text
#This will make it easier to call - you'll see it below


CK = 'yj50FC54j6ONZqxS5IzGnLdbF'
CS = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu'
AK = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1'
AS = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

#By setting these values to true, our code will automatically wait as it hits its limits
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Now I'm going to set up a custom stream listener
# I'll inherit all of the properties of the tweepy StreamListener
# But, I'm going to play with one particular method...
class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        #Here is where I use the json library to load the twitter data
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
    
    
while True:
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        #This next line puts a bounding box roughly around Seattle/Tacoma.
        #You start in the southwest and then go to the northeast
        #The format is longitude, then latitude... 
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

#Note: As written, this will run indefinitely. Use the stop button stop it.
#How might you write in a loop to only get a certain number of tweets?

florencevincent
She did it, so can you!
Washington, USA

LLHitz
@iandenning85 I think ur cool curve has done this if it makes u feel any better https://t.co/4vJ2IRdR3r
Seattle, WA

kevinokeefe
Maybe @Retroist, @colinokeefe, @j_sulz and @EJWalters are right about people paying for niche focused, well done an… https://t.co/0yHp45szyr
Seattle, WA

lmxtledesma12
Kinda upset that I’m missing the rodeo this year :/
Bremerton, WA

JSKChavez_
“Silly muthafucka who raised you
A ni**a with a pussy how disgraceful
I have my hittas come and duck tape you
And y… https://t.co/bHFq8gD37d
University Place, WA

preinsko
#Maddow That BS moving trying to get USPS to raise rates on Amazon in my opinion was an impeachable offense! I hate… https://t.co/ROClJ5StoR
Sammamish, WA

epalicki
@AllNewDom @tresdcomics To be fair, I’m sure the REAL real reason is that the studio overspent on marketing the movie to academy voters.
Seattle, WA



KeyboardInterrupt: 

### If that all worked, you now have a listener that will pull tweets from a bounded area you define.

**Cool**. Well, I think so. But, even though we're now pulling tweets *from* a location, you aren't saving their spatial data... *quite yet*.

That's where the lab actually begins.

### Question 1: Where the tweets at?

Using the example code above **and** the hints below, start pulling the spatial information from the tweets in question. Create a 'file' that contains a tweet's author (account name), its text, and the location from which it came (in latitude and longitude). This 'file' can be in a number of formats (geojson, txt, csv, etc.). *I stronly recommend you use csv*.

Bear in mind, there are *a few* ways you can pull location information. You can find the [twitter api documentation here](https://developer.twitter.com/en/docs/tutorials/filtering-tweets-by-location).

Some tweets will come from a 'location' that is a named place. In order to handle those, you will need to geocode the information. The function below takes a string and returns latitude and longitude. Start there.

I'm going to import a few libraries. That's because I think they'll be highly useful for you. As is often the case with python, there are many ways to go about this - I am simply suggesting the way that I have found most easily comprehensible.

In [2]:
from geopy import geocoders
import tweepy
import csv
import json

global tweets
tweets = 0
counter = 0

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def WriteCSV(user, text, lat, long):
    global tweets
    f = open('tweets.csv', 'a')
    write = csv.writer(f)
    write.writerow([user, text, lat, long])
    f.close()

import json

CK = 'yj50FC54j6ONZqxS5IzGnLdbF'
CS = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu'
AK = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1'
AS = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        WriteCSV(Author, text, x,y)

while True:
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

KeyboardInterrupt: 

### Question 2: Tweets on a map.

Now that you have a 'file' (or a script that will extract author, text, and location from tweets), let's make a map.

Using Folium, ArcGIS API for Python, GeoPandas, or Arcpy, create a map from your file. Make sure you accumulate enough tweets (let's say 100 or so) before you create the map.

Next week, we'll get into how to update the map on the fly and make it more interactive; for now, just make sure you can query some tweets, parse the data, put that data into a GIS of some form.

In [1]:
import pandas

tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

,User,Tweet,lat,long
0,susistuart,@BrettHamil We all know it's not supposed to s...,47.603832,-122.330062
1,CurtisM40613460,@SportsRadioKJR @johncanzanobft @cliffavril @J...,47.603832,-122.330062
2,Nmartin55,@NFCCHAMPS18 Yup. Its a zoo,47.685957,-122.192025
3,AliWantsFood,@DavidVonderhaar To make the older call of dut...,47.313494,-122.339310
4,lilith2u,@mariecountryman @KittyBhagat @renemjclarke @b...,47.603832,-122.330062


In [5]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas

tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

panda = pandas.DataFrame(tweets, columns=['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

tweetmap = folium.Map(
                location=[47.2529, -122.4443],
                zoom_start=8,
                tiles='OpenStreetMap')

tweetpoints = folium.features.GeoJson(geoPoints.to_json()).add_to(tweetmap)

tweetmap

A brief note on data binding: Although you are (most likely) using the Streaming API, the data in your maps is 'static.' What I mean is that once you create the map, you don't add data to it. There are ways to add data on the fly. In Esri environments, you do so with the [GeoEvent Server](http://www.esri.com/arcgis/products/geoevent-server). In other environments, you can use javascript frameworks such as [angular.js](https://angularjs.org/). Feel free to *experiment* with either. I might look at these guides [here](https://codehandbook.org/creating-a-web-app-using-angularjs-python-mongodb/) and [here](https://medium.com/@peregringaret/a-different-stack-angular-flask-mongodb-780b44e10afd) which use [flask](http://flask.pocoo.org/) and [mongodb](https://www.mongodb.com/) to create a web 'stack.'

Is all of that too much? No worries! That's why it's optional. Some of you are going to dive into python and swim within its majesty; others of you are going to learn the basics in order to create and deploy specific solutions or to test particular questions. **Either or both are fine!** The point of this class has never been to make you a programmer, but rather to teach you to *think computationally*, so that you can pursue programming and automation *as far as you need to*. You **can** go out and learn Flask on your own if you need to now, and that's awesome!

Ok, with that out of the way, let's get onto your next question...

### Question 3 - Cluster Map


Write a script that prompts the user for a keyword. Then, it monitors that keyword until 100 geocodable tweets have been gathered. Finally, it creates and saves an html map (the name is whatever keyword was chosen - \[keyword].html where the tweets are clustered at small scales and, at large scales, are points that when clicked upon Give the username and text of the tweet.

This is *similar* to the solution to **Question 2**; however, it adds a number of additional tasks.

In [1]:
from folium.plugins import MarkerCluster
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas
from geopy import geocoders
import tweepy
import csv
import json

#attempt at user input and csv saving
'''keywords = input('Enter a keyword to search for: ')

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def ClusterCSV(user, text, lat, long):
    global tweets
    f = open('cluster.csv', 'a')
    write = csv.writer(f)
    write.writerow([user, text, lat, long])
    f.close()

CK = 'yj50FC54j6ONZqxS5IzGnLdbF'
CS = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu'
AK = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1'
AS = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        ClusterCSV(Author, text, x,y)

while True:
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(track=[keywords], languages=["en"])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue'''

#Put a map on it!
tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

panda = pandas.DataFrame(tweets, columns=['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

tweetclustermap = folium.Map(
                location=[47.2529, -122.4443],
                zoom_start=8,
                tiles='OpenStreetMap')

#tweetpoints = folium.features.GeoJson(geoPoints.to_json()).add_to(tweetmap)

# convert to (n, 2) nd-array format for clustermap
tweetcluster = tweets[['lat', 'long']].values
# plot MarkerCluster
tweetclustermap.add_child(folium.plugins.MarkerCluster(tweetcluster))

tweetclustermap

In [10]:
tweetclustermap.save('grp4Lab5Cluster.html')
#tweetclustermap.save('['keyword'].html')

### Question 4 - Heatmap

Acknowledging that heatmaps, especially non-normalized ones, are [bogus](https://xkcd.com/1138/), they can still be cool to make. So, take your script above (asks for a keyword, monitors for that keyword until 100 entries have been geocoded, makes a map) and this time instead of clustering points, create a heatmap and save it as \[keyword].html.


In [2]:
from folium.plugins import HeatMap
import pandas

import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, geopandas

#attempt at user input and csv saving
'''keywords = input('Enter a keyword here: ')

def geo(location):
    g = geocoders.Nominatim()
    loc = g.geocode(location)
    return loc.latitude, loc.longitude

def HeatCSV(user, text, lat, long):
    global tweets
    f = open('heat.csv', 'a')
    write = csv.writer(f)
    write.writerow([user, text, lat, long])
    f.close()

CK = 'yj50FC54j6ONZqxS5IzGnLdbF'
CS = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu'
AK = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1'
AS = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        Data = json.loads(data)
        Author = Data['user']['screen_name']
        text = Data['text']
        print(Author)
        print(text)
        print(Data['place']['full_name'])
        print()
        x,y = geo(Data['place']['full_name'])
        HeatCSV(Author, text, x,y)

while True:
    try:
        
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        stream.filter(track=[keywords], languages=["en"])
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue'''

#Put a map on it!
tweets = pandas.read_csv('tweets.csv', encoding = 'ISO-8859-1')
tweets.head()

panda = pandas.DataFrame(tweets, columns=['User','Tweet','lat','long'])
#print(panda)

coords = zip(panda['long'], panda['lat'])

geometry = [shapely.geometry.Point(c) for c in coords]

geolist = geopandas.GeoSeries(geometry)

geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry=geolist)

geoPoints.crs=fiona.crs.from_epsg(4236)

tweetheatmap = folium.Map(
                location=[47.2529, -122.4443],
                zoom_start=8,
                tiles='OpenStreetMap')

#tweetpoints = folium.features.GeoJson(geoPoints.to_json()).add_to(tweetmap)

# convert to (n, 2) nd-array format for heatmap
tweetheat = tweets[['lat', 'long']].values

# plot heatmap
tweetheatmap.add_child(folium.plugins.HeatMap(tweetheat, radius=15))

tweetheatmap

In [13]:
tweetheatmap.save('grp4Lab5Heat.html')
#tweetheatmap.save('['keyword'].html')

### An impressive, difficult, and fascinating bonus question, + 4 pts, partial credit possible

Some of you used [NLTK](http://www.nltk.org/) to analyze our old friend H.P. Lovecraft. If you didn't, that's fine. NLTK is, as the name suggests, a Natural Language processing ToolKit. It's not the only one, and you are free to find and use another one, but I recommend it for this task.

You know those tweets? Who cares *how many* there are, let's talk about our ***feelings***. What you need to do now is run a sentiment analysis on your tweets. Categorize them by positive or negative emotions **and then create a heat map of how people feel according to their tweets**. 

Your task:

Create an interactive heatmap where the colors correspond not to number of tweets, but overall emotion of tweets from that area. In other words, interpolate according to the results of your sentiment analysis. While you can choose your own color ramp, I might recommend something like red for positive and blue for negative.

This must be based on **at least** 1,000 tweets.


In [3]:
#Twitter search plugin that we found that might work better

from TwitterSearch import *
try:
    tso = TwitterSearchOrder() #create a TwitterSearchOrder object
    tso.set_keywords(['bernie']) #define all words to search for
    tso.set_language('en') #laguage
    tso.set_include_entities(False) #and don't give all those entity information

    #secret tokens
    ts = TwitterSearch(
        consumer_key = 'yj50FC54j6ONZqxS5IzGnLdbF',
        consumer_secret = '1CDAlx6bzMO9XDw31NOLFk11vTFWaG0z1bG2i33k6Co3n2XBzu',
        access_token = '19347325-XIq3vbfAE8ZARoBEmXtmyHINnEVoFDu2nO90WgFF1',
        access_token_secret = 'GjDd67wr650A1GrPk5uZYtyXUlSxqjvbLx86wfqYQgk34'
     )

     #search
    for tweet in ts.search_tweets_iterable(tso):
        print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e: #errors
    print(e)

@IrishPatty54 tweeted: @KarenSm96080 @Jessica718784 @JustMehdi777 @thehill @SenSanders Get lost...no don’t bother I’ll just block you like I will all Bernie Bros
@renatelsj tweeted: RT @LeftwardSwing: Get it done! If you're in a state with closed primaries, make 100% sure you're registered as a Dem so you can vote for B…
@kay7017 tweeted: RT @PamelaGeller: Anti-Semite Bernie Sanders takes another shot at the Presidency: Jewish Americans should be very concerned about the pros…
@HawleysJadefav tweeted: RT @mitchellvii: So far the Democrats have about 8 different versions of Bernie but where is their Hillary?
@ResisterBc tweeted: RT @lynnv378: Ahem. Winning the primary &amp; general election depends on what????? Getting .@HillaryClinton's voters to vote for you.  So here…
@smallangryelf tweeted: RT @JordanUhl: This is wild.

Kamala first 24 hours: $1.5 million
Klobuchar first 48 hours $1 million
Warren first 24 hours: $300K

Bernie…
@vivafalastin tweeted: bernie should hire me then he’ll

@WildflowerSRQ tweeted: RT @andrewkimmel: Kamala’s first 24 hours: $1.5 million
Klobuchar’s first 48 hours $1 million
Warren’s first 24 hours: $300K

Bernie’s firs…
@LeslielvsB tweeted: RT @MrDane1982: Dear new followers,

I hate fucking Bernie Sanders. I will be vetting Bernie Sanders 24/7. So spare your little bernie feel…
@howardmclainjr tweeted: RT @flyer1618: "Honey, Bernie done broke out his box and is terrorizing the neighborhood again!"

#maga @potus #qanon #TheGreatAwakening #m…
@ConservaPeach tweeted: RT @LaylaAlisha11: 'Send in the clowns': Florida voters don't hold back on Bernie Sanders 2020 announcement! Voters say Socialists are sell…
@thepurplegrotto tweeted: RT @LisaMarieBoothe: Bernie Sanders, a socialist, is no longer outside of the mainstream of the Democrat Party. That is terrifying.
@GARussell tweeted: RT @LATiffani1: “I never had any money my entire life” - Bernie Sanders 

This incompetent communist who sold his soul out to wretched trai…
@shockatgambling tweete

@_justanopinion tweeted: RT @charliekirk11: Bernie Sanders spent nearly $300,000 on private air travel in October

For a committed socialist he sure seems to love l…
@jm_creighton tweeted: RT @guardian: Why Bernie Sanders' radicalism can take out Trump | Nathan Robinson https://t.co/mnIVvnfTDm
@AlisonBliss tweeted: RT @tomaskenn: Bernie sanders has so far raised 3.3 million dollars since his announcement at 7am this morning. I gave my $27. I’m so proud…
@cblunt48 tweeted: @TheRickWilson @runPattirun Worse than Bernie in many ways. But still a Russian stalking horse.
@ShermEstes tweeted: @Mhambi_M Nobody is controlling the narrative. It's like the Rs when they talk about 'MSM.' We're now in an age whe… https://t.co/DAFXtGMneK
@thewizardofawe tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@parterre tweeted: @ericsmithw @northeast_mama @fakedansavage And whether their first name is "

@sweetcharadejen tweeted: RT @PhilippeReines: Hillary Clinton, 70
Won 2016 Popular Vote by 2,864,903
Won 2016 Primaries by 3,708,294
Media: go away.

Bernie Sanders,…
@janeinak tweeted: @WajahatAli I am a big Bernie fan. He was not afraid to talk about health care as a right and student loans as an a… https://t.co/r3fiSGoekD
@khphanekham tweeted: Fine, Bernie. But I hope you’re actually vetted this time. And scrutinized about how you eat chicken and use hot sa… https://t.co/CGV2OJYOS9
@whilykit tweeted: I just contributed to Bernie Sanders https://t.co/zJMlc1PRqZ
@andymalum tweeted: RT @fathertamaki: I'll be able to vote next election! And guess what?? I'm not blindly voting for bernie! Nor for a specific political part…
@exec915 tweeted: RT @SecureAmerica: Bernie Sanders is running for President, but he supports lifting sanctions on the terrorist regime of Iran! #2020Dems

L…
@3ridersNOVA tweeted: @mcatty_alex Here comes the "but"! What if it turns out, there really is no collusion fr

@JohnRus58815368 tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@BobKski tweeted: @qanon76 It's like a twisted version of the Miss America pageant! I just hope that they don't do a swimsuit competi… https://t.co/JynDzZpXdB
@RobertRokdawg tweeted: RT @JakeMc945: It's the same Bernie Sanders as 2016 except he has 1 more house and sports car. Oh, and he wants to tax working folks 70% in…
@Jophus3 tweeted: Bernie Sanders is not trending? Horseshit.
@markly50 tweeted: @MollyJongFast @GOPChairwoman Waiting for her to attack Bernie or Tulsi
@tdv1414 tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@grots65 tweeted: RT @TheDCVince: Good old Breadline Bernie https://t.co/y7kt14cdjr
@a_harmonicminor tweeted: RT @SpacedOutRy: I try and stay away from talking about politics especially on e

...

@leesology tweeted: just about every tweet I’ve seen about Bernie $$$ has responses like “it came from Russians” and lol y’all I hate t… https://t.co/dNMNTJSxcN
@remy_4me tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@haylesatan tweeted: RT @armandodkos: Imagine a Dem primary where Harris captures 46% of delegates, Bernie 27% and the rest of the candidates 27%. Imagine the 2…
@NCdogs4Hillary tweeted: @GabbieKDrice Bernie has been so great for the state of Vermont that...they will pay you $10,000 to move there. 🤣… https://t.co/89jO1UcRdA
@TooProlix tweeted: @_strabbit @TeriPoulton Schultz was a member of the Democratic party. Bernie never was.
@anastasiakeeley tweeted: RT @ryangrim: Interested to hear what Jeff Sessions' spokesperson -- sorry, I mean CNN's 2020 political editor -- thinks of Bernie Sanders'…
@HopefulSemantic tweeted: @QanonAnonymous Yeah but how many babies has Ber

@WoJiangYou tweeted: RT @ggreenwald: One of the big, looming questions of 2020, especially with Bernie's announcement today, is what @AOC - a devoted Berne camp…
@charlynn_rick tweeted: RT @PhilippeReines: Hillary Clinton, 70
Won 2016 Popular Vote by 2,864,903
Won 2016 Primaries by 3,708,294
Media: go away.

Bernie Sanders,…
@shelibeli007 tweeted: RT @RyanAFournier: Socialism has claimed the lives of millions of people. There is no GOOD version of socialism that Bernie and Cortez keep…
@CarolynStanto15 tweeted: RT @mmpadellan: Now that Bernie Sanders has announced running for President in 2020, I wonder if the media will ask:

Is he likeable?
But h…
@Bnayakno1Md tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@SarcasticBride tweeted: RT @msleen1970: It’s so funny how there’s people shielding for Bernie &amp; have no problem with him running again. While @HillaryClinton is be…
@Dobr

@kevinco74585206 tweeted: RT @SebGorka: .@TheDemocrats have found the youthful face of the future. 

https://t.co/yxUVusq7Lm
@Love_MyLife tweeted: RT @ClaraJeffery: A very early test for Bernie is if he and his surrogates speak out, loudly and repeatedly, against supporters doling out…
@braveryraptor tweeted: RT @scottdetrow: Just in from Bernie Sanders’ campaign: he’s raised $3.3 million so far today - so, double that Kamala Harris first day hau…
@NinaMar10031107 tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@Damndrew2 tweeted: RT @AbbyMartin: Since Bernie announced his run, the corporate media is falling over themselves to denounce his massively popular policies a…
@libstopher tweeted: RT @Esor_Fasa: The reality is that Bernie Sanders is the candidate who will best improve the material conditions for the bottom 90% of peop…
@Nayda02053763 tweeted: RT @slickeryslick: @RealJamesW

@SBSF8084 tweeted: RT @PuestoLoco: .#NeverBernie
Don’t let Bernie Cult shame you into passing on his age by saying “Ageism” is a bigotry. It’s NOT. Voters hav…
@ckhopkar tweeted: The Bernie Bros out in force today reminding everyone how to alienate potential allies and create division. Well done comrades!
@bugmayer6 tweeted: RT @ambertamblyn: If Bernie Sanders wins the Democratic nomination, I will do what most should have done for Hillary Clinton in the 2016 el…
@AngelaCimmaruta tweeted: I’m perplexed by these people who keep trying to make Bernie Sanders happen. I’m still pissed at the way he conduct… https://t.co/kVLeFIYlJK
@dana43284013 tweeted: RT @charliekirk11: Obama and Clinton have gotten away with: 

Uranium One
Benghazi
Solyndra
Fast &amp; Furious
IRS targeting
Cash to Iran
Riggi…
@Julkirk tweeted: RT @RealMattCouch: Why any of you idiots would vote for Bernie Sanders is beyond me.. 

This guy literally let the Clinton's and the DNC sc…
@Jennifer_Manley tweeted: RT @ambertam

KeyboardInterrupt: 